<a href="https://colab.research.google.com/github/karoz-exe/Homework-Kazancev-R.A-/blob/hw_6/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%966.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Задание к практической работе №6:**

В этом задании вам предстоит использовать данные о выживаемости на Титанике для построения модели бинарной классификации для признака Survived.

* Считайте датасет из файла train.csv (это данные о выживаемости на Титанике)
* Выберите и обоснуйте метрику для измерения качества (accuracy/precision/recall/f1-score/fbeta-score/roc-auc и т.д.). В рамках данного пункта необходимо подобрать наиболее релевантную метрику или набор метрик для вашей задачи, написав краткое обоснование (1-2 предложения) - 2 балла
* Постройте бейзлайн и ML-модель классификации (LogisticRegression или любая другая, которая вам кажется подходящей) и оцените их качество с помощью выбранной метрики

**Критерии оценки модели:**

* Произведено разбиение датасета на тренировочную/тестовую выборки - 1 балл
* Произведено измерение качества константного предсказания (например, наиболее частотный класс/случайное предсказание) - 1 балл
* ML-модель обучена на тренировочной выборке, учтены особенности предобработки данных для модели, если они есть - 3 балла
* Произведено измерение качества на отложенной выборке с использованием ранее выбранной метрики - 1 балл
* Обеспечена воспроизводимость решения: зафиксированы random_state, ноутбук воспроизводится от начала до конца без ошибок - 2 балла

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
df = pd.read_csv("/content/sample_data/data titanic.csv")
# Подзадание №1. Выбор метрики: В задаче предсказания выживаемости на Титанике важно минимизировать количество непредсказанных выживших.  Исходя из этого, выбрана метрика Recall.
df = df.dropna(subset=['Embarked', 'Age']) #Удаляем строки с пропусками в этих столбцах
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
X = df.drop(['PassengerId', 'Survived', 'Name', 'Ticket', 'Cabin'], axis=1)
y = df['Survived']
# Разбиение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Бейзлайн: предсказание наиболее частого класса
most_frequent_class = y_train.mode()[0]
y_pred_baseline = np.full(y_test.shape, most_frequent_class)
print("Метрики бейзлайна:")
print(y_pred)
print(np.bincount(y_test))
print("\nМетрики Logistic Regression:")
print(classification_report(y_test, y_pred, zero_division=0))
# ML-модель: Logistic Regression
numeric_features = ['Age', 'SibSp', 'Parch', 'Fare','Embarked']
categorical_features = ['Sex','Pclass']

numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(random_state=42))])

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("\nМетрики Logistic Regression:")
print(classification_report(y_test, y_pred))

print(f"Accuracy: {accuracy_score(y_test,y_pred)}")
print(f"Recall: {recall_score(y_test,y_pred)}")
print(f"Precision: {precision_score(y_test,y_pred)}")
print(f"F1-score: {f1_score(y_test,y_pred)}")

try:
    print(f"ROC AUC: {roc_auc_score(y_test, y_pred)}")
except ValueError as e:
    print(f"Ошибка при вычислении ROC AUC: {e}") #Обработка потенциальной ошибки при бинарной классификации с только одним классом

Метрики бейзлайна:
[1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 1 1 0 1 1 0 0 1 0 1 0 1 1 0 0 0 0 0 0
 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0
 0 0 1 0 0 1 0 1 1 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 1 1 1 0 1 0 0 0 0 0 0 0 0
 1 0 0 1 0 0 0 1 1 0 1 1 1 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0]
[80 63]

Метрики Logistic Regression:
              precision    recall  f1-score   support

           0       0.77      0.90      0.83        80
           1       0.84      0.65      0.73        63

    accuracy                           0.79       143
   macro avg       0.80      0.78      0.78       143
weighted avg       0.80      0.79      0.79       143


Метрики Logistic Regression:
              precision    recall  f1-score   support

           0       0.77      0.90      0.83        80
           1       0.84      0.65      0.73        63

    accuracy                           0.79       143
   macro avg       0.80      0.78      0.78       143
weighted avg       0.80